### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [3]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/226_TROCAS7_Fusion_Apr2021_PEAKS_156/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/226_TROCAS7_Fusion_Apr2021_PEAKS_156


In [4]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [5]:
ls 

 226_TROCAS7_Fusion_Apr2021_PEAKS_156_peptide.csv
 226_TROCAS7_Fusion_Apr2021_PEAKS_156_protein.csv
 226_TROCAS7_Fusion_Apr2021_PEAKS_156_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [6]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup226 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/226_TROCAS7_Fusion_Apr2021_PEAKS_156/226_TROCAS7_Fusion_Apr2021_PEAKS_156_peptide.csv")

# remove redundant rows
peaksdb226 = pd.DataFrame.drop_duplicates(peaksdbdup226)

print(peaksdb226.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb226.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb226['#Spec']
del peaksdb226['Accession']
del peaksdb226['PTM']
del peaksdb226['AScore']

mean_length = peaksdb226['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup226))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb226))

#look at the dataframe
peaksdb226.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-226-NMCP', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-226-NMCP', 'Accession', 'PTM', 'AScore'],
      dtype='object')
10.927710843373495
# redundant peaksdb peptides in combined dataframe 83
# nonredundant peaksdb peptides in combined dataframe 83


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,83.66,1881.8350,18,1.1,941.9258,91.47,1380000.0,63,16361,20210407_Trocas7_665_NMCP226_DDA_120min_1.raw
1,LGEHNIDVLEGNEQFINAAK,78.74,2210.0967,20,1.3,737.7071,95.87,3650000.0,63,17117,20210407_Trocas7_665_NMCP226_DDA_120min_1.raw
2,IDVLEGNEQFINAAK,69.23,1659.8468,15,1.8,830.9322,94.62,2110000.0,63,16902,20210407_Trocas7_665_NMCP226_DDA_120min_1.raw
3,SSGSSYPSLLQC(+57.02)LK,58.57,1525.7446,14,0.8,763.8802,99.66,300000.0,63,17744,20210407_Trocas7_665_NMCP226_DDA_120min_1.raw
4,VLEGNEQFINAAK,52.77,1431.7357,13,1.5,716.8762,71.89,1970000.0,63,13056,20210407_Trocas7_665_NMCP226_DDA_120min_1.raw


In [7]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb226['A'] = peaksdb226['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb226['C'] = peaksdb226['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb226['D'] = peaksdb226['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb226['E'] = peaksdb226['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb226['F'] = peaksdb226['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb226['G'] = peaksdb226['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb226['H'] = peaksdb226['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb226 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb226['I'] = peaksdb226['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb226['K'] = peaksdb226['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb226['L'] = peaksdb226['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb226['M'] = peaksdb226['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb226['N'] = peaksdb226['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb226['P'] = peaksdb226['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb226['Q'] = peaksdb226['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb226['R'] = peaksdb226['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb226['S'] = peaksdb226['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb226['T'] = peaksdb226['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb226['V'] = peaksdb226['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb226['W'] = peaksdb226['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb226['Y'] = peaksdb226['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb226['c-carb'] = peaksdb226['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb226['m-oxid'] = peaksdb226['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb226['n-deam'] = peaksdb226['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb226['q-deam'] = peaksdb226['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb226['stripped_peptide'] = peaksdb226['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb226['stripped_length'] = peaksdb226['stripped_peptide'].apply(len)

peaksdb226['NAAF_num.'] = peaksdb226['Area'] / peaksdb226['stripped_length']

# total the number of modifications in sequence
peaksdb226['ptm-total'] = peaksdb226['c-carb'] + peaksdb226['m-oxid'] + peaksdb226['n-deam'] + peaksdb226['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb226['stripped_IL']= peaksdb226['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb226.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/226_MN_T00_GD_PDB/226_MN_T00_GD_PDB.csv")

# check out the results
peaksdb226.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,83.66,1881.8350,18,1.1,941.9258,91.47,1380000.0,63,16361,...,0,2,0,0,0,SCLISGWGNTK,11,125454.545455,2,SCLLSGWGNTK
1,LGEHNIDVLEGNEQFINAAK,78.74,2210.0967,20,1.3,737.7071,95.87,3650000.0,63,17117,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,182500.000000,0,LGEHNLDVLEGNEQFLNAAK
2,IDVLEGNEQFINAAK,69.23,1659.8468,15,1.8,830.9322,94.62,2110000.0,63,16902,...,0,0,0,0,0,IDVLEGNEQFINAAK,15,140666.666667,0,LDVLEGNEQFLNAAK
3,SSGSSYPSLLQC(+57.02)LK,58.57,1525.7446,14,0.8,763.8802,99.66,300000.0,63,17744,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,21428.571429,1,SSGSSYPSLLQCLK
4,VLEGNEQFINAAK,52.77,1431.7357,13,1.5,716.8762,71.89,1970000.0,63,13056,...,0,0,0,0,0,VLEGNEQFINAAK,13,151538.461538,0,VLEGNEQFLNAAK


In [8]:
# keep only stripped peptide I/L and NAAF
db_226 = peaksdb226[['stripped_IL', 'Area', 'NAAF_num.']]

db_226.set_index('stripped_IL')

# write modified dataframe to new txt file
db_226.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/226_MN_T00_GD_PDB_ILnaafs.csv")

db_226.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,1380000.0,125454.545455
1,LGEHNLDVLEGNEQFLNAAK,3650000.0,182500.000000
2,LDVLEGNEQFLNAAK,2110000.0,140666.666667
3,SSGSSYPSLLQCLK,300000.0,21428.571429
4,VLEGNEQFLNAAK,1970000.0,151538.461538


### Exporting txt files of stripped peptides at confidence cutoffs:

In [9]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep226moddup = peaksdb226[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep226dup = peaksdb226[["stripped_peptide"]]

# deduplicate both of these lists
pep226mod = pep226moddup.drop_duplicates()
pep226 = pep226dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 226:", len(pep226moddup))
print("Deduplicated modified PeaksDB peptides in 226:", len(pep226mod))
print("Total stripped PeaksDB peptides in 226:", len(pep226dup))
print("Deduplicated stripped PeaksDB peptides in 226:", len(pep226))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep226.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/226_MN_T00_GD_PDB/226_MN_T00_GD_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/226_MN_T00_GD_PDB/226_MN_T00_GD_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/226_MN_T00_GD_PDB/226_MN_T00_GD_PDB_stripped_peptides.fas

# look at the stripped peptides
pep226.head()

Total modified PeaksDB peptides in 226: 83
Deduplicated modified PeaksDB peptides in 226: 83
Total stripped PeaksDB peptides in 226: 83
Deduplicated stripped PeaksDB peptides in 226: 81


,stripped_peptide
0,SCLISGWGNTK
1,LGEHNIDVLEGNEQFINAAK
2,IDVLEGNEQFINAAK
3,SSGSSYPSLLQCLK
4,VLEGNEQFINAAK


## NAAF correction and exporting files with AA and PTM totals:

In [10]:
# made a new dataframe that contains the suMN of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb226['A'].sum(),
        'C': peaksdb226['C'].sum(),
        'D': peaksdb226['D'].sum(),
        'E': peaksdb226['E'].sum(),
        'F': peaksdb226['F'].sum(),
        'G': peaksdb226['G'].sum(),
        'H': peaksdb226['H'].sum(),
        'I': peaksdb226['I'].sum(),
        'K': peaksdb226['K'].sum(),
        'L': peaksdb226['L'].sum(),
        'M': peaksdb226['M'].sum(),
        'N': peaksdb226['N'].sum(),
        'P': peaksdb226['P'].sum(),
        'Q': peaksdb226['Q'].sum(),
        'R': peaksdb226['R'].sum(),
        'S': peaksdb226['S'].sum(),
        'T': peaksdb226['T'].sum(),
        'V': peaksdb226['V'].sum(),
        'W': peaksdb226['W'].sum(),
        'Y': peaksdb226['Y'].sum(),
        'c-carb': peaksdb226['c-carb'].sum(),
        'm-oxid': peaksdb226['m-oxid'].sum(),
        'n-deam': peaksdb226['n-deam'].sum(),
        'q-deam': peaksdb226['q-deam'].sum(),
        'Total area': peaksdb226['Area'].sum(),
        'Total length': peaksdb226['stripped_length'].sum()
       }

totalpeaksdb226 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb226['% C w/ carb'] = totalpeaksdb226['c-carb'] / totalpeaksdb226['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb226['% M w/ oxid'] = totalpeaksdb226['m-oxid'] / totalpeaksdb226['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb226['% N w/ deam'] = totalpeaksdb226['n-deam'] / totalpeaksdb226['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb226['% Q w/ deam'] = totalpeaksdb226['q-deam'] / totalpeaksdb226['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb226['NAAF denom.'] = totalpeaksdb226['Total area'] / totalpeaksdb226['Total length']

# write modified dataframe to new txt file
totalpeaksdb226.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/226_MN_T00_GD_PDB/226_MN_T00_GD_PDB_totals.csv")

totalpeaksdb226.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,83,13,29,48,33,67,22,72,63,109,...,4,10,2,487919690.0,887,1.0,0.571429,0.106383,0.058824,550078.568207


In [11]:
# use the calculated NAAF factor (in totalpeaksdb226 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 550078.568207

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb226['NAAF factor'] = (peaksdb226['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksdb226_NAAF = peaksdb226[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb226_NAAF['A-NAAF20'] = peaksdb226_NAAF['A'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['C-NAAF20'] = peaksdb226_NAAF['C'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['D-NAAF20'] = peaksdb226_NAAF['D'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['E-NAAF20'] = peaksdb226_NAAF['E'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['F-NAAF20'] = peaksdb226_NAAF['F'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['G-NAAF20'] = peaksdb226_NAAF['G'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['H-NAAF20'] = peaksdb226_NAAF['H'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['I-NAAF20'] = peaksdb226_NAAF['I'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['K-NAAF20'] = peaksdb226_NAAF['K'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['L-NAAF20'] = peaksdb226_NAAF['L'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['M-NAAF20'] = peaksdb226_NAAF['M'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['N-NAAF20'] = peaksdb226_NAAF['N'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['P-NAAF20'] = peaksdb226_NAAF['P'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['Q-NAAF20'] = peaksdb226_NAAF['Q'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['R-NAAF20'] = peaksdb226_NAAF['R'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['S-NAAF20'] = peaksdb226_NAAF['S'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['T-NAAF20'] = peaksdb226_NAAF['T'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['V-NAAF20'] = peaksdb226_NAAF['V'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['W-NAAF20'] = peaksdb226_NAAF['W'] * peaksdb226['NAAF factor']
peaksdb226_NAAF['Y-NAAF20'] = peaksdb226_NAAF['Y'] * peaksdb226['NAAF factor']

# multiply the NAAF20 factor by the PTMN normalize its abundance by peak area and peptide length

peaksdb226_NAAF['ccarb-NAAF20'] = peaksdb226_NAAF['c-carb'] * peaksdb226_NAAF['NAAF factor']
peaksdb226_NAAF['moxid-NAAF20'] = peaksdb226_NAAF['m-oxid'] * peaksdb226_NAAF['NAAF factor']
peaksdb226_NAAF['ndeam-NAAF20'] = peaksdb226_NAAF['n-deam'] * peaksdb226_NAAF['NAAF factor']
peaksdb226_NAAF['qdeam-NAAF20'] = peaksdb226_NAAF['q-deam'] * peaksdb226_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb226_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/226_MN_T00_GD_PDB/226_MN_T00_GD_PDB_naaf.csv")

peaksdb226_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,0.228067,3,2,0,1,0,3,0,1,...,0.0,0.456133,0.456133,0.000000,0.228067,0.000000,0.456133,0.0,0.0,0.0
1,LGEHNIDVLEGNEQFINAAK,0.331771,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,0.331771,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,IDVLEGNEQFINAAK,0.255721,2,0,1,2,1,1,0,2,...,0.0,0.000000,0.000000,0.255721,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,SSGSSYPSLLQCLK,0.038955,0,1,0,0,0,1,0,0,...,0.0,0.194777,0.000000,0.000000,0.000000,0.038955,0.038955,0.0,0.0,0.0
4,VLEGNEQFINAAK,0.275485,2,0,0,2,1,1,0,1,...,0.0,0.000000,0.000000,0.275485,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [12]:
# made a new dataframe that contains the suMN of NAAF normalized AAs for peaksdb226 peaksdb results
# also contains the suMN of the NAAF-corrected PTMN occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb226_NAAF['NAAF factor'].sum(),
        'A': peaksdb226_NAAF['A-NAAF20'].sum(),
        'C': peaksdb226_NAAF['C-NAAF20'].sum(),
        'D': peaksdb226_NAAF['D-NAAF20'].sum(),
        'E': peaksdb226_NAAF['E-NAAF20'].sum(),
        'F': peaksdb226_NAAF['F-NAAF20'].sum(),
        'G': peaksdb226_NAAF['G-NAAF20'].sum(),
        'H': peaksdb226_NAAF['H-NAAF20'].sum(),
        'I': peaksdb226_NAAF['I-NAAF20'].sum(),
        'K': peaksdb226_NAAF['K-NAAF20'].sum(),
        'L': peaksdb226_NAAF['L-NAAF20'].sum(),
        'M': peaksdb226_NAAF['M-NAAF20'].sum(),
        'N': peaksdb226_NAAF['N-NAAF20'].sum(),
        'P': peaksdb226_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb226_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb226_NAAF['R-NAAF20'].sum(),
        'S': peaksdb226_NAAF['S-NAAF20'].sum(),
        'T': peaksdb226_NAAF['T-NAAF20'].sum(),
        'V': peaksdb226_NAAF['V-NAAF20'].sum(),
        'W': peaksdb226_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb226_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb226_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb226_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb226_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb226_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb226_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb226_NAAF['% C w/ carb.'] = totalpeaksdb226_NAAF['c-carb'] / totalpeaksdb226_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb226_NAAF['% M w/ oxid'] = totalpeaksdb226_NAAF['m-oxid'] / totalpeaksdb226_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb226_NAAF['% N w/ deam'] = totalpeaksdb226_NAAF['n-deam'] / totalpeaksdb226_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb226_NAAF['% Q w/ deam'] = totalpeaksdb226_NAAF['q-deam'] / totalpeaksdb226_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb226_NAAF['NAAF check'] = totalpeaksdb226_NAAF['NAAF'] / 550078.568207

# write modified dataframe to new txt file, same name + totals
totalpeaksdb226_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/226_MN_T00_GD_PDB/226_MN_T00_GD_PDB_naaf_totals.csv")

totalpeaksdb226_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,104.482449,71.068392,1.783708,3.852153,5.978002,3.3076,10.187003,6.047889,37.412737,7.463493,...,1.08321,1.783708,0.061655,1.732843,0.68013,1.0,0.019532,0.07722,0.076167,0.00019
